In [1]:
import re
import pandas as pd

from toolz.functoolz import excepts
from urllib.request import urlretrieve

In [2]:
import sys; sys.path.append('../data/')

Все необходимые импорты есть в файле `bacchus.py`. Поэтому достаточно импортировать его:

In [3]:
import bacchus

----

# Загрузка данных

Загрузим данные и убедимся, что все на месте:

In [4]:
urlretrieve ("http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls", "titanic.xls");

In [5]:
df = pd.read_excel('titanic.xls')
df.shape

(1309, 14)

In [6]:
df.head(3)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


# Подготовка данных

С визуализацией и анализом исходных данных можно ознакомиться в ряде статей:
* https://habrahabr.ru/post/274171/
* https://habrahabr.ru/company/mlclass/blog/270973/

В этой тетрадке я буду вдаваться в подробности того, почему генерируем конкретные признаки именно так, а не иначе. Вместо этого буду просто отталкиваться от задач, которые будут постепенно перед нами вставать, и как их решить с помощью __Bacchus__. 

## Генерация новых признаков

Начнем с простого: у нас есть один признак, например, полное имя пассажира, а нам хочется извлечь оттуда его титул. 

Создадим лямбда-выражение, которое умеет выделить его из строки:

In [7]:
extract_title = lambda s: s.split(',')[1].split('.')[0].strip()

Чтобы вставить его в пайплайн, поможет обертка, которая называется просто: __Transformer__. 

Применить его можно следующим образом:

__```Transformer```__```(```__```lambda```__```, [```__```apply_on```__```, [```__```value_name```__```]])```

Предполагается два варианта использования:

* `Transformer(lambda)`. В данном случае lambda будет применена ко всему __DataFrame__.
* `Transformer(lambda, apply_on[, value_name])`. В этом варианте предполагается, что lambda принимает уже только единственный столбец, то есть __Series__. Если параметр `value_name` не указан, то новый столбец будет называться "`value`". 

Значит, вставить трансформер в пайплайн можно так:

In [8]:
_ = ('title',  bacchus.Transformer(extract_title, apply_on='name', value_name='title'))

Продолжим развивать идею: новые лямбды, новые признаки:

In [9]:
extract_ticket_number = excepts(Exception, lambda x: int(re.findall(r'(\d+)', str(x))[0]), lambda _: -1)
extract_ticket_prefix = excepts(Exception, lambda x: re.findall(r'([a-zA-Z\.\s]+)', str(x))[0], lambda _: '')

In [10]:
cabin_class = excepts(Exception, lambda s: s[0], lambda _: np.nan)
cabin_side  = excepts(Exception, lambda s: int(re.findall(r'\d+', s)[0]) % 2,  lambda _: -1)

Наконец, это можно объединить:

In [11]:
pipeline = bacchus.DFPipeline([
    ('title',  bacchus.Transformer(extract_title,         apply_on='name',   value_name='title')),
    ('number', bacchus.Transformer(extract_ticket_number, apply_on='ticket', value_name='ticket_number')),
    ('prefix', bacchus.Transformer(extract_ticket_prefix, apply_on='ticket', value_name='ticket_prefix')),
    ('class',  bacchus.Transformer(cabin_class,           apply_on='cabin',  value_name='cabin_class')),
    ('side',   bacchus.Transformer(cabin_side,            apply_on='cabin',  value_name='cabin_side'))
])

In [12]:
df_new_features = pipeline.fit_transform(df)

Убеждаемся, что новые признаки добавлены:

In [13]:
df_new_features.head(1)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,title,ticket_number,ticket_prefix,cabin_class,cabin_side
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",Miss,24160,,B,1


## Более сложный кейс

Как известно, нельзя впихнуть невпихуемое в однострочную лямбду.

Безусловно, вы можете написать целую функцию и захватить ее __Transformer__'ом. ~~Но зачем?~~

Для более продвинутой логики рекомендуется создать наследника от класса AbstractTransformer, и работать уже с ним.

Обратившись к признаку `title` можно видеть, что некоторые титулы вроде "Don", "Sir", "Capt" встречаются крайне редко, и вряд ли несут много полезной информации. Их вполне можно переименовать к "каноническим" вроде "Mr".

Давайте посмотрим, как это можно решить: 

In [14]:
class TitleRenaimer(bacchus.AbstractTransformer):
    def __init__(self, column_to_apply_on, **other):
        super().__init__(**other)
        self.column_to_apply_on = column_to_apply_on
    
    def transform(self, X, **other):
        replacements = {
            'Dr': 'Mr', 'Rev': 'Mr', 'Col': 'Mr', 'Ms': 'Miss', 'Major': 'Mr', 'Mlle': 'Miss', 
            'the Countess': 'Mrs', 'Capt': 'Mr', 'Dona': 'Mrs', 'Sir': 'Mr', 'Mme': 'Mrs', 
            'Lady': 'Mrs', 'Jonkheer': 'Mr', 'Don': 'Mr'
        }
        X[self.column_to_apply_on] = X[self.column_to_apply_on].apply(lambda x: replacements[x] if x in replacements else x)
        return X

In [15]:
df_new_features.title.value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Dr                8
Rev               8
Col               4
Major             2
Mlle              2
Ms                2
the Countess      1
Lady              1
Capt              1
Don               1
Mme               1
Dona              1
Sir               1
Jonkheer          1
Name: title, dtype: int64

In [16]:
df_correct_title = TitleRenaimer(column_to_apply_on='title').fit_transform(df_new_features)

In [17]:
df_correct_title.title.value_counts()

Mr        783
Miss      264
Mrs       201
Master     61
Name: title, dtype: int64

## Работа с пропусками в данных

Для работы с пропусками в данных предусмотрен __FillNaTransformer__. 

Мы отошли от `sklearn`-реализации класса __Imputer__: 
* хотелось заполнять пропуски по-разному одним и тем же унифицированным интерфейсом;
* не хватало группирования по значениям некоторого другого признака.

Смотрим, что получилось?

Вот какие варианты для заполнения предлагает __FillNaTransformer__:
* __`mean`__ - заполняет средним значением.
* __`mode`__ - заполняет модой.
* __`median`__ - заполняет медианой.
* __`interpolate`__ - линейно интерполирует пропуски.
* __`akima`__ - интерполяция с помощью [AKIMA](http://stackoverflow.com/a/4626304).
* __`ffill`__ - аналогично с `ffill` оригинальной реализации Sklearn.
* __`pad`__ - аналогично с `pad` оригинальной реализации Sklearn.
* __`bfill`__ - аналогично с `bfill` оригинальной реализации Sklearn.
* __`backfill`__ - аналогично с `backfill` оригинальной реализации Sklearn.
* __`fill`__ - аналогично с `fill` оригинальной реализации Sklearn.

Синтаксис следующий: мы кормим __FillNaTransformer__'у словарь, показывающий компуктору, что делать в каждой из колонок с пропусками.

Формат: 
```
{ 
    <имя колонки>: <что делать с пропусками>,
    <имя колонки>: <что делать с пропусками>,
    ...
}
```

Варианты `<что делать с пропусками>`:
* __Название метода__. См. выше.
* __Словарь__. Должен содержать 2 ключа: `method` и `groupby`. Очевидно, применяет заполнение с выбранным методом внутри группы значений другого признака.
* __Конкретное значение__. Можно написать любую строку или число, и оно, как константа, поставляется в каждый из пропусков. 

Давайте быстрее к примеру! Благо в "Титанике" можно продемонстрировать все случаи:

In [18]:
fna = bacchus.FillNaTransformer(columns_strategies={
    # Методы
    'embarked': 'mode',         # мода
    'fare': 'median',           # медиана

    # Метод с объединением
    'age': dict(method='mean',  # пропуск значения у женщины будет заполнено средним возрастом женщины на борту,
                groupby='sex'), #  а пропуск значения мужчины - средним возрастом мужчины

    # Конкретные значения
    'boat': 'unknown',
    'cabin_class': 'unknown',
    'home.dest': 'unknown',
    'cabin': 'unknown',
    'body': 'unknown'
})

In [19]:
df_without_nas = fna.fit_transform(df_correct_title)

In [20]:
df_without_nas.isnull().any()

pclass           False
survived         False
name             False
sex              False
age              False
sibsp            False
parch            False
ticket           False
fare             False
cabin            False
embarked         False
boat             False
body             False
home.dest        False
title            False
ticket_number    False
ticket_prefix    False
cabin_class      False
cabin_side       False
dtype: bool

## Кодирование признаков

Мы решили не ходить далеко, а сделать обертку над __`category_encoders`__, которая по сути не только содержит наиболее часто используемые способы кодирования из __`sklearn`__, но и предлагает ряд своих.

Всего в вашем распоряжении 8 способов кодировать признаки:
* __`onehot`__ 
* __`binary`__
* __`backward`__
* __`ordinal`__
* __`sum`__
* __`poly`__
* __`helmert`__
* __`hash`__

Достаточно просто указать, какие колонки мы хотим закодировать в параметре `columns_include`:

In [21]:
encoding_pipeline = bacchus.DFPipeline([
    ('label_encode', bacchus.CustomEncoder('ordinal', columns_include=['sex', 'embarked', 'boat', 
                                                                       'cabin_class', 'ticket_prefix'])),      
    ('onehot_encode', bacchus.CustomEncoder('onehot', columns_include=['title'])),
])

In [22]:
df_encoded = encoding_pipeline.fit_transform(df_without_nas)

## Для чего мы все это затевали?

Конечно, чтобы объединить все сразу!

Наш итог, все вместе на текущий момент:

In [23]:
pipeline = bacchus.DFPipeline([
    ('title',  bacchus.Transformer(extract_title, apply_on='name', value_name='title')),
    ('number', bacchus.Transformer(extract_ticket_number, apply_on='ticket', value_name='ticket_number')),
    ('prefix', bacchus.Transformer(extract_ticket_prefix, apply_on='ticket', value_name='ticket_prefix')),
    ('cabin_class', bacchus.Transformer(cabin_class, apply_on='cabin', value_name='cabin_class')),
    ('cabin_side', bacchus.Transformer(cabin_side, apply_on='cabin', value_name='cabin_side')),
    
    ('correct_titles', TitleRenaimer(column_to_apply_on='title')),
    
    ('fill_na', bacchus.FillNaTransformer(columns_strategies={
        'age': dict(method='mean',
                    groupby='sex'),
        'embarked': 'mode',
        'fare': 'median',
        'boat': 'unknown',
        'cabin_class': 'unknown',
        'home.dest': 'unknown'
    })),
    
    ('label_encode', bacchus.CustomEncoder('ordinal', columns_include=['sex', 'embarked', 'boat', 
                                                                       'cabin_class', 'ticket_prefix'])),      
    ('onehot_encode', bacchus.CustomEncoder('onehot', columns_include=['title'])),
    
    ('drop_useless', bacchus.Transformer(lambda df: df.drop(['body', 'name', 'ticket', 'cabin', 'home.dest'], axis=1))),
], verbose=True)

Обратите, кстати, на флажок `verbose`: с его помощью можно оценить, как меняется размер датафрейма в процессе преобразований, а также находить самые медленные шаги.

In [24]:
result = pipeline.fit_transform(df)

DFPipeline               2017-04-03 17:01:00.745747	(1309, 14)
Transformer              2017-04-03 17:01:00.750393	(1309, 15)
Transformer              2017-04-03 17:01:00.755390	(1309, 16)
Transformer              2017-04-03 17:01:00.760672	(1309, 17)
Transformer              2017-04-03 17:01:00.764133	(1309, 18)
Transformer              2017-04-03 17:01:00.769032	(1309, 19)
TitleRenaimer            2017-04-03 17:01:00.770014	(1309, 19)
FillNaTransformer        2017-04-03 17:01:00.798551	(1309, 19)
CustomEncoder            2017-04-03 17:01:01.021808	(1309, 19)
CustomEncoder            2017-04-03 17:01:01.045142	(1309, 22)
Transformer              2017-04-03 17:01:01.046703	(1309, 17)


Мы готовы к работе: все наши данные - числовые, и пропусков нет.

In [25]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 17 columns):
title_0          1309 non-null int64
title_1          1309 non-null int64
title_2          1309 non-null int64
title_3          1309 non-null int64
pclass           1309 non-null int64
survived         1309 non-null int64
sex              1309 non-null int64
age              1309 non-null float64
sibsp            1309 non-null int64
parch            1309 non-null int64
fare             1309 non-null float64
embarked         1309 non-null int64
boat             1309 non-null int64
ticket_number    1309 non-null int64
ticket_prefix    1309 non-null int64
cabin_class      1309 non-null int64
cabin_side       1309 non-null int64
dtypes: float64(2), int64(15)
memory usage: 173.9 KB
